In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%writefile requirements.txt

tensorflow
keras
split-folders
tqdm
termcolor
colorama
sklearn

Overwriting requirements.txt


In [3]:
%%writefile iqc_train_model.py

import argparse
import os
import shutil

import numpy as np
import splitfolders
from tqdm import trange
from sklearn.utils import shuffle

from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, AveragePooling2D, BatchNormalization
from keras.preprocessing.image import load_img, img_to_array

WD = os.getcwd()
CLASSES = ['bad', 'average', 'good']

def create_data():
    # creating a new folder for every class
    if not os.path.exists('data'):
        os.mkdir('data')
    for c in CLASSES:
        if not os.path.exists(os.path.join('data', c)):
            os.mkdir(os.path.join('data', c))

    # saving all scores in a list
    scores = []
    with open('drive/MyDrive/scores.txt', 'r') as sc:
        for record in sc.readlines():
            scores.append(record.split(',')[1].rstrip('\n'))

    # copying the image to the corresponding class folder based on score
    images = len(os.listdir('drive/MyDrive/frames'))
    if len(os.listdir('data/good')) == 0 and len(os.listdir('data/average')) == 0 and len(os.listdir('data/bad')) == 0:
        for image in trange(images):
            shutil.copy(f'drive/MyDrive/frames/{image}.png', f'data/{scores[image]}')

    # splitting into train, test && validation
    splitfolders.ratio('data', output="split_data", seed=1337, ratio=(.8, .1, .1))

def preprocess_data():
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []

    # resize, grayscale, define labels
    for c in CLASSES:
        for img in os.listdir(f'split_data/train/{c}'):
            train_images.append(img_to_array(load_img(f'split_data/train/{c}/' + img, target_size=(256, 256)))[:, :, 0])
            train_labels.append(c)

    for c in CLASSES:
        for img in os.listdir(f'split_data/test/{c}'):
            test_images.append(img_to_array(load_img(f'split_data/test/{c}/' + img, target_size=(256, 256)))[:, :, 0])
            test_labels.append(c)

    # reshape, normalize, one color channel
    train_images = np.array(train_images).reshape(len(train_images), 256, 256, 1) / 255
    train_labels = np.array(train_labels)
    test_images = np.array(test_images).reshape(len(test_images), 256, 256, 1) / 255
    test_labels = np.array(test_labels)

    # convert train labels to numerical
    for i in range(len(train_labels)):
        train_labels[i] = CLASSES.index(train_labels[i])
    train_labels = train_labels.astype('int32')

    # convert test labels to numerical
    for i in range(len(test_labels)):
        test_labels[i] = CLASSES.index(test_labels[i])
    test_labels = test_labels.astype('int32')

    train_images, train_labels = shuffle(train_images, train_labels)
    test_images, test_labels = shuffle(test_images, test_labels)

    return train_images, train_labels, test_images, test_labels

def build_model(train_images, train_labels, test_images, test_labels):
    model = keras.Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(256, 256, 1)))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'], optimizer='adam')
    model.fit(train_images, train_labels, epochs=8, batch_size=128)

    model.summary()
    model.evaluate(test_images, test_labels, batch_size=64)

    return model

if __name__ == '__main__':
    print('This might take a while..')

    # organize images & split folders
    create_data()
    # preprocess the images
    X_train, y_train, X_test, y_test = preprocess_data()
    # build the model and train it
    
    print('Training..')
    model = build_model(X_train, y_train, X_test, y_test)

    # save model
    filepath = './image_quality_classifier'
    keras.models.save_model(model, filepath)
    print('Model saved!')

Overwriting iqc_train_model.py


In [4]:
%%writefile iqc_predict.py

import argparse
import os
import shutil
from termcolor import *
import colorama
colorama.init()

import numpy as np
import splitfolders
from tqdm import trange
from keras.preprocessing.image import load_img, img_to_array
from tensorflow import keras

def arg_parser():
    ap = argparse.ArgumentParser()
    ap.add_argument("-i", "--image", required=True, help="input an image")

    return vars(ap.parse_args())

def load_model():
    filepath = 'image_quality_classifier'
    model = keras.models.load_model(filepath, compile=True)
    return model

def preprocess_input(img):
    sample = np.array(img_to_array(load_img(img, target_size=(256, 256)))[:, :, 0])
    sample = sample.reshape(1, sample.shape[0], sample.shape[1], 1)
    sample /= 255
    return sample

if __name__ == '__main__':
    output_quality = {0: 'bad', 1: 'average', 2:'good'}
    model = load_model()
    args = arg_parser()
    img = preprocess_input(args['image'])
    predictions = model.predict(img)
    print('Image quality:')
    if np.argmax(predictions, axis=1)[0] == 0:
        cprint(output_quality[np.argmax(predictions, axis=1)[0]], 'red')
    elif np.argmax(predictions, axis=1)[0] == 1:
        cprint(output_quality[np.argmax(predictions, axis=1)[0]], 'yellow')
    elif np.argmax(predictions, axis=1)[0] == 2:
        cprint(output_quality[np.argmax(predictions, axis=1)[0]], 'green')

Overwriting iqc_predict.py


In [ ]:
!pip install -q -r requirements.txt

In [6]:
!python3 iqc_train_model.py

This might take a while..
100% 1002/1002 [04:33<00:00,  3.67it/s]
Copying files: 1002 files [00:02, 428.59 files/s]
Training..
2021-11-21 22:54:44.971474: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Epoch 1/8
7/7 [==============================] - 36s 566ms/step - loss: 4.3695 - accuracy: 0.3333
Epoch 2/8
7/7 [==============================] - 3s 415ms/step - loss: 0.9489 - accuracy: 0.3308
Epoch 3/8
7/7 [==============================] - 3s 416ms/step - loss: 0.6968 - accuracy: 0.5768
Epoch 4/8
7/7 [==============================] - 3s 416ms/step - loss: 0.5035 - accuracy: 0.6667
Epoch 5/8
7/7 [==============================] - 3s 416ms/step - loss: 0.4033 - accuracy: 0.6679
Epoch 6/8
7/7 [==============================] - 3s 415ms/step - loss: 0.3578 - accuracy: 0.7653
Epoch 7/8
7/7 [==============================] - 3s 417ms/step - loss: 0

Predictions on the 'frames' folder:

In [13]:
!python3 iqc_predict.py --image drive/MyDrive/frames/0.png

2021-11-21 22:56:30.306427: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Image quality:
bad


In [14]:
!python3 iqc_predict.py --image drive/MyDrive/frames/6.png

2021-11-21 22:56:37.025739: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Image quality:
average


In [15]:
!python3 iqc_predict.py --image drive/MyDrive/frames/1.png

2021-11-21 22:56:43.946051: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Image quality:
good


Predictions on images in the validation folder:

In [10]:
!python3 iqc_predict.py --image split_data/val/bad/120.png

2021-11-21 22:56:09.029700: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Image quality:
bad


In [11]:
!python3 iqc_predict.py --image split_data/val/average/115.png

2021-11-21 22:56:16.116563: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Image quality:
average


In [12]:
!python3 iqc_predict.py --image split_data/val/good/111.png

2021-11-21 22:56:22.724733: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Image quality:
good
